<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h3><font size=5>Parte 2 - Modelos de Regresión</font></h3>
<br>
<h1><font size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>



<div align="right">
<font size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>

---

In [34]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Array manipulation libraries
import numpy as np
import pandas as pd

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Simple regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR, SVR

# Bagging ensemble models
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

# Boosting ensemble models
from sklearn.ensemble import AdaBoostRegressor, HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Experimentation
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
# Seed for random experiments - 7 is the number
RANDOM_SEED = 777

En la primera libreta se realizó un **análisis exploratorio de datos** exhaustivo para entender en profundidad el comportamiento del conjunto de datos de interés - el **segundo desafío** del **Women in Data Science (*WiDS*) Datathon** del año 2024, disponible en el [siguiente enlace](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/overview).

Tras este estudio, el objetivo de la siguiente libreta es tanto la **construcción de modelos de regresión** capaces de predecir el **tiempo de diagnóstico de la metástasis** a partir de los atributos seleccionados, como la **evaluación** de estos con el fin de estudiar si resultan de utilidad y si - como se planteó - los **atributos geográficos, socioeconómicos y climáticos** juegan algún papel relevante en la estimación de los valores.

---

# Índice

* [3. Selección de atributos y pre-procesamiento](#section3)
    * [3.1. Carga y particionamiento del conjunto de datos](#section3-1)
    * [3.2. Selección de atributos](#section3-2)
    * [3.3. Pre-procesamiento de los atributos seleccionados](#section3-3)
* [4. Selección de modelos de regresión e hiperparámetros](#section4)
* [5. Experimentación](#section5)
* [6. Análisis de resultados](#section6)
* [7. Conclusiones](#section7)
---

<a id="section3"></a>

# 3. Selección de atributos y pre-procesamiento

Tras finalizar el **análisis exploratorio de datos** en la libreta anterior, el siguiente paso en el proceso de ciencia de datos es el **preprocesamiento de la información** - para ser utilizada posteriormente por modelos de regresión, con el fin de predecir el tiempo de diagnóstico de la metástasis.

Concretamente, en este apartado se realizan las siguientes preparaciones:
- **Cargar y particionar** los conjuntos de datos en **entrenamiento**, **validación** y **test**.
- **Seleccionar el subconjunto de atributos** que van a ser utilizados durante la experimentación.
- **Preparar las *pipelines*** encargadas de transformar los datos crudos en datos listos para ser utilizados por los modelos posteriores.

---

<a id="section3-1"></a>

## 3.1. Carga y particionamiento del conjunto de datos

Durante el análisis exploratorio de datos se trabajó únicamente sobre el **conjunto de entrenamiento** - con el fin de evitar cualquier posible fuga de datos al estudiar el conjunto de test. Ahora bien, el desafio en Kaggle ofrece **dos conjuntos de datos**:
- `train.csv`: El **conjunto de entrenamiento**, con **150 atributos** y los valores de la **variable objetivo** (el tiempo de diagnóstico) asociados a cada instancia.
- `test.csv`: El **conjunto de test**, conteniendo únicamente los **150 atributos** sin los valores de la variable objetivo.

El primer paso, por tanto, consiste en **cargar ambos conjuntos de datos** como *DataFrames*:

In [3]:
# Loading the CSV files - force zip3 to be read as a string
df_train = pd.read_csv("data/train.csv", index_col="patient_id", dtype={"patient_zip3": object})
df_test = pd.read_csv("data/test.csv", index_col="patient_id", dtype={"patient_zip3": object})

# Display a small sample of both datasets to show that they have been properly loaded
print(f"Training size: {df_train.shape}")
display(df_train.sample(5))
print(f"Test size: {df_test.shape}")
display(df_test.sample(5))

Training size: (13173, 151)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
449386,Asian,MEDICARE ADVANTAGE,CA,948,West,Pacific,65,F,NaN,C50919,...,55.67,58.21,66.64,72.31,68.68,67.19,63.51,57.23,49.34,0
794302,White,MEDICAID,LA,706,South,West South Central,61,F,NaN,C50911,...,63.74,77.51,81.80,83.07,82.46,80.32,71.56,56.24,53.39,69
598518,NaN,NaN,CA,934,West,Pacific,70,F,NaN,C50411,...,59.45,60.24,64.77,69.81,70.13,68.10,65.38,60.72,54.08,49
111010,NaN,COMMERCIAL,IN,463,Midwest,East North Central,69,F,23.0,1749,...,40.53,66.83,71.29,73.14,73.34,67.87,51.74,34.15,32.06,260
869218,White,NaN,PA,152,Northeast,Middle Atlantic,63,F,NaN,C50919,...,46.61,68.51,71.31,75.37,74.94,71.16,55.81,39.52,36.31,41


Test size: (5646, 150)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
patient_id,,,,,,,,,,,,,,,,,,,,,
902542,Black,COMMERCIAL,MI,483,Midwest,East North Central,65,F,25.0,C50119,...,30.47,38.10,62.27,66.66,70.12,70.65,63.73,47.91,32.25,30.68
164714,Other,MEDICAID,CA,923,West,Pacific,72,F,NaN,C50212,...,49.93,59.55,63.23,73.00,80.90,78.71,72.55,60.09,52.12,45.17
967406,Asian,NaN,CA,921,West,Pacific,75,F,NaN,C50812,...,58.44,62.22,62.69,67.56,75.29,75.89,70.88,67.22,63.06,56.38
784208,White,COMMERCIAL,CA,915,West,Pacific,50,F,NaN,C50912,...,59.08,63.13,63.24,67.79,76.33,75.32,71.25,68.59,64.83,59.14
219479,Other,COMMERCIAL,VA,234,South,South Atlantic,28,F,NaN,1748,...,43.90,55.31,71.53,76.06,77.05,79.05,76.88,63.12,49.75,44.80


Ahora bien, debido al proceso que se va a seguir durante el entrenamiento de los modelos (**selección de hiperparámetros**, **selección de modelos** y **evaluación**), utilizar directamente los conjuntos de datos descritos podría llevar a un problema de **fuga de datos** - al usar el mismo conjunto de datos para entrenar los modelos y evaluar sus hiperparámetros.

Para evitar esto, se va a dividir el conjunto de entrenamiento en dos - un conjunto de **entrenamiento** y uno de **validación**  -, siendo la distribución final la siguiente:
- **Entrenamiento**: El conjunto de entrenamiento cumple dos tareas - tanto el **entrenamiento de los modelos de regresión** propuestos como el **ajuste de hiperparámetros de los mismos** a través de una validación cruzada.
- **Validación**: Una vez se tienen los modelos entrenados, el conjunto de validación será utilizado para **seleccionar el mejor modelo de forma honesta** - utilizando un conjunto de datos que no han utilizado durante el entrenamiento para evitar sesgos o fugas de datos.
- **Test**: Finalmente, se utilizará el conjunto de test para **evaluar el rendimiento real** del modelo seleccionado a través de la validación - utilizando una plataforma externa (***Kaggle***) para medir este rendimiento. 

Además, todos estos conjuntos de datos se van a fraccionar en **atributos** (`X`) y **variable objetivo** (`y`) para seguir el estándar de `scikit-learn`.

In [4]:
# Split the datasets into training, validation and test
# Train / Val
X, y = df_train.drop(columns="metastatic_diagnosis_period"), df_train["metastatic_diagnosis_period"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

# Test
X_test = df_test

# Display the information about each dataset - to ensure that it has been loaded and partitioned correctly
print("ENTRENAMIENTO:")
print(f"\t-Atributos: {X_train.shape}")
print(f"\t-Variable objetivo: {y_train.shape}")
print("VALIDACIÓN:")
print(f"\t-Atributos: {X_val.shape}")
print(f"\t-Variable objetivo: {y_val.shape}")
print("TEST:")
print(f"\t-Atributos: {X_test.shape}")

ENTRENAMIENTO:
	-Atributos: (9879, 150)
	-Variable objetivo: (9879,)
VALIDACIÓN:
	-Atributos: (3294, 150)
	-Variable objetivo: (3294,)
TEST:
	-Atributos: (5646, 150)


---

<a id="section3-2"></a>

## 3.2. Selección de atributos

Como se observó durante el análisis exploratorio, no tendría sentido utilizar directamente el **conjunto de datos completos** para el entrenamiento de modelos:
- La **dimensionalidad del conjunto de datos** - con 150 atributos en total - es excesiva para la cantidad de datos disponible, lo que podría llevar a sobreajustes.
- Algunos atributos tienen **una cantidad excesiva de posibles valores** - que se puede traducir, de nuevo, en sobreajustes del modelo al no tener suficientes datos para aprender adecuadamente las relaciones.
- La **amplia mayoría de atributos son irrelevantes** para la variable objetivo - ya sea por su baja calidad o por la poca correlación que tienen con la variable objetivo.

Por tanto, es necesario realizar una **selección de un subconjunto de atributos** para reducir la dimensionalidad y cribar los atributos que no sean relevantes para la predicción. Para buscar este subconjunto, se proponen varias opciones:

### 3.2.1. Selección por análisis exploratorio

Durante el análisis exploratorio de datos se ha realizado un análisis exhaustivo de los datos - tanto su **comportamiento** como su **relevancia** y las **transformaciones** que serían necesarias para utilizarse.

A partir de las conclusiones extraidas, se ha obtenido el siguiente **conjunto de atributos** - representando los atributos más relevantes estudiados dentro del conjunto de datos, junto a las **transformaciones a aplicar** sobre éstos:
- **Código de diagnóstico del cancer de mama (`breast_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Código de diagnóstico del cancer metastático (`metastatic_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Estado de residencia del paciente (`patient_state`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Raza del paciente (`patient_race`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.
- **Tipo de seguro médico del paciente (`payer_type`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor.

In [5]:
categorical_manual = [
    "breast_cancer_diagnosis_code",
    "metastatic_cancer_diagnosis_code",
    "patient_race",
    "payer_type",
    "patient_state"
]

Estas transformaciones se han elegido en base a los **test estadísticos** que se realizaron durante el análisis exploratorio:
- La **agrupación de los valores** en las variables de alta dimensionalidad aumenta la significación estadística, al reducirse el número de valores con un número demasiado bajo de instancias.
- La **sustitución de valores perdidos** mejora el rendimiento en los atributos donde el número de valores perdidos es excesivo.

A su vez, se ha optado por descartar los siguientes atributos:
- **Edad (`patient_age`) y IMC (`bmi`) del paciente**: Variables numéricas sin correlación con la variable objetivo.
- **Región (`Region`) y división (`Division`) del paciente**: Variables categóricas con poca relevancia, y ya representadas por otra variable más significativa (`patient_state`).
- **Código zip del paciente (`patient_zip3`)**: Variable categórica ya representada por otra variable (`patient_state`) con dimensionalidad excesiva.
- **Todas las variables geográficas, socioeconómicas y climáticas**: 136 atributos numéricos sin correlación con la variable objetivo.

### 3.2.2. Selección automática

En el apartado anterior se ha obtenido un subconjunto de atributos en base al estudio que se realizó en la libreta anterior. Ahora bien, aunque el estudio estuviese apoyado en **gráficas y tests estadísticos**, sigue existiendo la posibilidad de que **el analista haya introducido sesgos propios**.

Otra posibilidad es utilizar **algoritmos de aprendizaje automático** para realizar el proceso de selección de atributos - ya sea basandose en **tests estadísticos** o en el **comportamiento de modelos reales entrenados sobre los datos**.

Ahora bien, la mayoría de métodos necesitan un **preprocesamiento previo**, para imputar valores perdidos y unificar el comportamiento de atributos categóricos y numéricos. Concretamente, en este caso:
- **Atributos categóricos**: Se imputan los valores perdidos como un valor constante (`UNKNOWN`) y se **codifican los posibles valores del atributo** utilizando `One-Hot Encoding` - donde cada valor de cada variable categórica se **representa como un atributo separado**, que puede ser `True` (si el valor de la variable se corresponde) o `False` en cualquier otro caso.
    - Debido a la gran complejidad de algunos atributos, se opta por **agrupar todos los valores con menos de 100 instancias**. Este valor se ha elegido de forma arbitraria para la selección de variables, y tendrá que ser ajustado posteriormente durante el ajuste de hiperparámetros.
    - La codificación va a causar que **aumente el número de atributos respecto al conjunto de entrenamiento original** - lo que significa que **será necesario re-agrupar los valores de las variables mediante estadísticos** para realizar la selección.
- **Atributos numéricos**: Se imputan los valores perdidos utilizando la **mediana** (debido al gran número de valores extremos) y se **escalan los valores** utilizando la mediana y la desviación estándar.

Se define un `Pipeline` para realizar automáticamente el preprocesamiento descrito - aplicandose sobre los datos del **conjunto de entrenamiento** para evitar fugas de datos:

In [6]:
# Column types
categorical_variables = X_train.select_dtypes(exclude=np.number).columns.to_list()
numerical_variables = X_train.select_dtypes(include=np.number).columns.to_list()

# Feature Selection pipeline - Transforms the dataset to allow for feature selection
# NOTE: No categorical variable grouping is performed - which may lead to a massive number of attributes

# Categorical transformations - imputing and one-hot encoding
fs_categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
    ("ohencoder", OneHotEncoder(min_frequency=100))
])

# Numerical transformations - imputing and scaling
fs_numerical = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Final pre-processing
fs_preprocessing = ColumnTransformer([
    ("cat", fs_categorical, categorical_variables),
    ("num", fs_numerical, numerical_variables)
])

Existen dos estrategias para realizar una selección automática de atributos:

#### - Métodos *filter*

Los **métodos de filtrado (*filter*)** utilizan **test estadísticos** para evaluar de forma automática la relevancia de cada variable, sin la necesidad de entrenar modelos- haciendolos **más agiles**, aunque menos capaces de identificar las correlaciones entre grupos de atributos.

Para este caso se utilizan **tests F** - tests de **varianza**, para identificar las **diez variables** más relevantes del conjunto de datos 

In [7]:
# FEATURE RANKING - Using F-Regression
# Create the pipeline
fs_kbest = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("kbest", SelectKBest(f_regression, k="all"))
])
fs_kbest.fit(X_train, y_train)

# Join the variable names and values
df_kbest = pd.DataFrame({
    "variable": fs_kbest.get_feature_names_out(),
    "score": fs_kbest["kbest"].scores_
})

# Process the dataframe, step by step
df_kbest_ordered = (
    df_kbest.assign(variable=(
        df_kbest["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")   # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_kbest_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_desc,3163.104702,293.539683
breast_cancer_diagnosis_code,3163.104702,293.539683
metastatic_cancer_diagnosis_code,58.998175,12.662202
payer_type,56.152011,21.747065
patient_age,32.205066,32.205066
breast_cancer_diagnosis_desc_infrequent,30.799057,30.799057
breast_cancer_diagnosis_code_infrequent,30.799057,30.799057
patient_state,28.378242,3.208365
labor_force_participation,15.166246,15.166246


A partir de estos atributos obtenidos, se observa que:
- Los **atributos más importantes** son consistentes con los observados durante el estudio: **el código de diagnóstico de cancer** - tanto original como metástasis - y el **tipo de seguro médico**.
- El algoritmo da **mayor importancia** a algunos atributos numéricos - principalmente:
    - **Edad del paciente** (`patient_age`)
    - **Porcentaje de residentes empleados** (`labor_force_participation`)
    - **Porcentaje de residentes con estudios universitarios** (`education_bachelors` y `education_college_or_above`)
    - **Porcentaje de hogares con dos o más ingresos** (`family_dual_income`)
- Algunos atributos categóricos estudiados **tienen menor importancia de la esperada**:
    - **Estado de residencia del paciente** (`patient_state`)
    - **Raza del paciente** (`patient_race`)

Es de interés destacar que **existe una diferencia muy considerable en la importancia de los atributos** - donde el **tipo de cancer de mama original** es varios ordenes de magnitud más relevante que el resto de variable.

El **subconjunto de variables** obtenido por el proceso es el siguiente:

In [8]:
categorical_filter = ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "payer_type", "patient_state", "patient_race"]
numerical_filter = ["patient_age", "labor_force_participation", "education_bachelors", "family_dual_income", "education_college_or_above"]

#### - Métodos *wrapper*

Los **métodos de envoltura *wrapper*** utilizan **modelos de aprendizaje automáticos** entrenados sobre los datos para elegir las variables más relevantes. Pese a ser más lentos, sus resultados son **más fiables** - al estar estudiando el comportamiento real de un modelo.

Para este caso se entrenará un modelo de **Random Forest**, utilizando el **conjunto de entrenamiento** para evitar fugas - buscando encontrar los **diez atributos más relevantes**:

In [9]:
# FEATURE RANKING - Using Random Forest as a wrapper
# Create the pipeline
fs_rf = Pipeline([
    ("preprocessing", fs_preprocessing),
    ("wrapper", RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED))
])
fs_rf.fit(X_train, y_train)

# Join the variable names and values
df_rf = pd.DataFrame({
    "variable": fs_rf["preprocessing"].get_feature_names_out(),
    "score": fs_rf["wrapper"].feature_importances_
})

# Process the dataframe, step by step
df_rf_ordered = (
    df_rf.assign(variable=(
        df_rf["variable"].str.extract(r"cat__(?P<cat>[0-9A-Za-z_\- ]+)_(?:[0-9A-Za-z\- ]+|infrequent)|num__(?P<num>[0-9A-Za-z_\- ]+)")      # 1 - Extract the proper variable name
        .apply(lambda s: s["cat"] if not pd.isna(s["cat"]) else s["num"], axis=1)                                                           #     (joining into a single column)
    ))
    .groupby("variable").agg(max_score=("score", "max"), avg_score=("score", "mean"))                                                       # 2 - Obtain the maximum and avg score of each variable
    .sort_values(by="max_score", ascending=False)                                                                                           # 3 - Sort by MAXIMUM score
    
)

# Display the 15 best attributes
display(df_rf_ordered.head(15))

,max_score,avg_score
variable,,
breast_cancer_diagnosis_code,0.123410,0.012368
breast_cancer_diagnosis_desc,0.119221,0.012102
patient_age,0.079427,0.079427
bmi,0.037954,0.037954
metastatic_cancer_diagnosis_code,0.012096,0.004549
breast_cancer_diagnosis_desc_infrequent,0.010563,0.010563
breast_cancer_diagnosis_code_infrequent,0.009509,0.009509
payer_type,0.007699,0.006449
patient_race,0.007403,0.005098


*(NOTA: `RandomForest` tiene un componente aleatorio. Se ha utilizado una semilla, pero sigue existiendo la posibilidad de que cambien los valores en ejecuciones posteriores)*

A partir de estos atributos, se observa que:

- En este caso, el atributo más importante es **únicamente el código de diagnóstico del cancer de mama** - el diagnóstico de metástasis **pierde peso**.
- Tienen un mayor peso los **atributos numéricos**:
    - **Edad** (`patient_age`) y **IMC** (`bmi`) del paciente.
    - **Mediana del tiempo de transporte de los residentes al trabajo** (`commute_time`).
    - **Porcentaje de la población con edad superior a 40 años** (`age_40s`).
    - **Porcentaje de la población con estudios en STEM** (`education_stem_degree`).
    - **Porcentaje de la población identificada con razas nativas** (`race_native`).
- Los **atributos categóricos identificados** - **raza del paciente** (`patient_race`) y **tipo de seguro médico** (`payer_type`) pierden importancia.
- **El estado de residencia del paciente** (`patient_state`) deja de estar entre los **diez atributos más relevantes**.

El subconjunto de atributos obtenidos por este proceso es el siguiente:

In [10]:
categorical_wrapper = ["breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code", "patient_race", "payer_type"]
numerical_wrapper = ["patient_age", "bmi", "commute_time", "race_native", "education_stem_degree", "age_40s"]

### 3.2.3. Conjunto de datos completo

Finalmente, otra opción posible es **utilizar todos los atributos del conjunto de datos** sin ningún tipo de criba o pre-selección. Esta opción tiene ciertas **ventajas** y **desventajas**:
- Por un lado, **algunos modelos funcionan mejor sin selección de variables** - especialmente aquellos que realizan una selección interna (como la **regresión lineal de tipo Lasso** o los **ensembles tipo Random Forest**).
- Por otro lado, **la complejidad excesiva del conjunto de datos** puede afectar al entrenamiento - provocando sobreajustes por la falta de instancias en el conjunto de datos, y perjudicando al rendimiento de los modelos sin selección de atributos interna. Además, **el tiempo de entrenamiento aumenta considerablemente** debido a la mayor complejidad de los modelos entrenados.

En este caso, se incluye principalmente como ***baseline*** para el resto de opciones - pudiendo de esta forma estudiar si **la selección de variables ha mejorado el rendimiento de los modelos**.

In [11]:
categorical_full = X_train.select_dtypes(exclude=np.number).columns.to_list()
numerical_full = X_train.select_dtypes(include=np.number).columns.to_list()

---

<a id="section3-3"></a>

## 3.3. Pre-procesamiento de los datos

El último paso antes del entrenamiento de los modelos es el **preprocesamiento del conjunto de datos** - realizar las transformaciones adecuadas sobre el conjunto de datos para **optimizarlo** de cara al entrenamiento y evaluación de los modelos, con el fin de mejorar su rendimiento.

Los pasos que se van a seguir para pre-procesar los datos - dependiendo del **tipo de datos** - son los siguientes:
- **Atributos categóricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por un **valor constante** - `UNKNOWN`.
        - Como se estudió durante el análisis exploratorio, en la mayoría de variables categóricas **tiene sentido tratar los valores perdidos como un valor distinto** - al poder representar valores que no se han podido adquirir o que no se han querido compartir.
    2. **Codificación**: Se transforma la representación de la variable utilizando **`One-Hot Encoding`** - el atributo se divide en **tantos atributos como valores tiene la variable**, donde cada uno de estos nuevos atributos representa si la instancia contiene el valor representado (`1`) o no (`0`).
        - Debido a la gran complejidad de los atributos categóricos (teniendo, en general, **40 o más posibles valores**) y a que **los atributos no son exhaustivos** y pueden haber valores no vistos antes en el conjunto de datos, es necesario **agrupar los valores menos frecuentes** para reducir la dimensionalidad. El **umbral de agrupamiento** será uno de los hiperparámetros a ajustar posteriormente.
- **Atributos numéricos**:
    1. **Imputación**: Se reemplazan todos los valores perdidos por **la mediana del atributo**.
        - En general, el conjunto de datos tiene un gran número de **valores extremos y sesgos**. Utilizar la mediana frente a la media ayuda a crear valores más robustos.
    2. **Escalado**: Se *centran* los valores de los atributos alrededor de la **mediana** y la **desviación estándar**.
        - Normalmente se escala utilizando el **valor promedio**. Ahora bien, para tener un escalado más robusto se utiliza el **rango intercuartil** para el escalado a través de la clase `RobustScaler`.

El problema ahora es que los pasos de preprocesamiento deben ser **idénticos y replicables** en todos los aspectos - tanto entre los experimentos con **distintos modelos y subconjuntos de atributos**, como entre **el entrenamiento y la evaluación** con distintos conjuntos de datos.

Para automatizar este proceso, se definen **`Pipelines`** - cadenas de transformaciones que se aplican **automáticamente** antes de usar los modelos de aprendizaje automático, tanto para **entrenarlos** como para **predecir la variable objetivo para un conjunto de datos**. Es necesario definir `Pipelines` para cada uno de los subconjuntos de atributos que se van a utilizar:

#### - Subconjunto manual

Al tener únicamente atributos categóricos, el `Pipeline` solo contiene las transformaciones asociadas a las variables categóricas. 

Ahora bien, para facilitar la automatización, se **utilizará la estructura dividida por tipos de dato** utilizada por el resto de *Pipelines* - mediante `ColumnTransformer`:

In [12]:
# MANUAL PIPELINE - Categorical values
# Reuses the ColumnTransformer structure to automatize accessing the OneHotEncoder hyperparameters - as min_frequency will be adjusted
# Otherwise, the code would have to take into account the subset of attributes being used to know the inner pipeline structure

preprocessing_manual = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_manual)
])
display(preprocessing_manual)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_race', 'payer_type',
                                  'patient_state'])])

### - Subconjuntos automáticos (*filter* y *wrapper*) y conjunto de datos completo

En estos casos, al tener atributos **categóricos** y **numéricos** el *Pipeline* necesita poder transformar por separado los dos tipos de atributos - utilizando `ColumnTransformer`:

In [13]:
# AUTOMATIC PIPELINE - Filter values
preprocessing_filter = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_filter),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_filter)
])
display(preprocessing_filter)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'payer_type', 'patient_state',
                                  'patient_race']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'labor_force_participation',
                                  'education_bachelors', 'family_dual_income',
                                  'education_college_or_above'])])

In [14]:
# AUTOMATIC PIPELINE - Wrapper
preprocessing_wrapper = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_wrapper),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_wrapper)
])
display(preprocessing_wrapper)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['breast_cancer_diagnosis_code',
                                  'metastatic_cancer_diagnosis_code',
                                  'patient_race', 'payer_type']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['patient_age', 'bmi', 'commute_time',
                                  'race_native', 'education_stem_degree',
                                  'age_40s'])])

In [15]:
# FULL PIPELINE
# AUTOMATIC PIPELINE - Wrapper
preprocessing_full = ColumnTransformer([
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("onehot", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=100))
    ]), categorical_full),
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())
    ]), numerical_full)
])
display(preprocessing_full)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='UNKNOWN',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=100))]),
                                 ['patient_race', 'payer_type', 'patient_state',
                                  'patient_zip3', 'Region', 'Division',
                                  'patient_gender',
                                  'breast_cancer_diagnosis_code',
                                  'breast_cancer_d...
                                  'age_60s', 'age_70s', 'age_over_80', 'male',
                                  'female', 'married', 'divorced',
                                  'never_married', 'widowed', 'family_size',
                                  'family_dual_income',
                                  'income_household_median',
                                  'income_household_under_5',
                                  'income_household_5_to_10',
                                  'income_household_10_to_15',
                                  'income_household_15_to_20',
                                  'income_household_20_to_25',
                                  'income_household_25_to_35',
                                  'income_household_35_to_50', ...])])

Para facilitar la automatización de las siguientes secciones y evitar la redundancia de código, se puede definir una **lista** conteniendo todas las fases de pre-procesamiento:

In [16]:
preprocessing_dict = {
    "manual": preprocessing_manual,
    "filter": preprocessing_filter,
    "wrapper": preprocessing_wrapper,
    "full": preprocessing_full
}

---

<a id="section4"></a>

# 4. Selección de modelos de regresión e hiperparámetros

En el apartado anterior se realizó la **selección de atributos** - proponiendo varios posibles subconjuntos de atributos para estudiar durante la experimentación - y se definió el **preprocesamiento** a realizar sobre estos atributos, de cara a prepararlos para el estudio.

El siguiente paso en el proceso de ciencia de datos, por tanto, sería el **entrenamiento y evaluación de modelos** a partir de los datos procesados y seleccionados. Ahora bien, antes de comenzar con el entrenamiento de los modelos, es necesario realizar una **selección de los modelos** que van a ser estudiados - junto a una **selección de hiperparámetros** a ser estudiado para cada modelo.

Por tanto, el objetivo de esta sección es:
1. Definir y estudiar los **estimadores** que se entrenaran y evaluarán durante el proceso de ciencia de datos.
    - Se propondrá una serie de modelos que servirán como *baseline* - el rendimiento base al que todos los modelos deberían aspirar a cumplir.
        - Sobre esto, se propondrán diversos modelos de regresión de familias típicas para los problemas de regresión - modelos **lineales** y **basados en árboles**.
    - Finalmente, se utilizarán ***ensembles*** - modelos estado del arte basados en agrupaciones de modelos más sencillos.
2. Seleccionar los **hiperparámetros** a estudiar para cada modelo.
    - Para ajustar el rendimiento de cada modelo al problema concreto, durante la experimentación se entrenará cada modelo varias veces probando distintos **conjuntos de hiperparámetros**.

Para simplificar la automatización, los modelos a entrenar serán almacenados en el diccionario `models_dict`, siguiendo la siguiente estructura:

```
models_dict[<nombre del modelo>]:
    "hyperparameters": {diccionario de hiperparametros},
    "models": {
        "manual": Pipeline,
        "filter": Pipeline,
        "wrapper": Pipeline,
        "full": Pipeline
    }
```

In [17]:
models_dict = {}


---

<a id="section4-1"></a>

## 4.1. *Baselines* - estimadores lineales y basados en árboles

En esta sección se definen los estimadores **simples** a utilizar durante la experimentación. Es importante distinguir que entendemos como **estimador simple / base** a cualquier estimador que **funciona de forma independiente** - frente a los *ensembles*, que funcionan como agrupaciones de estimadores simples. 

*(NOTA: Si bien la mayoría de modelos son a su vez sencillos a nivel de coste computacional, hay algunos (como las máquinas de vectores de soporte) que pueden tener costes elevados durante el entrenamiento. El abanico de posibles es más amplio del propuesto - se ha realizado una selección teniendo en cuenta el **tamaño del conjunto de datos** y la **alta dimensionalidad de este**.)*

En concreto, se van a estudiar algoritmos de las siguientes familias de modelos:
- **Lineales** - regresión lineal y algoritmos basados en ésta.
- **Arboles de decisión**.
- **Máquinas de vectores de soporte**.


El objetivo de los modelos propuestos en esta sección es servir como un *baseline* - una **puntuación base** que debería ser superada por el resto de modelos complejos, pero que sirve como un punto de referencia del **error promedio esperable si se entrenase el modelo más simple posible**.

---

### 4.1.1. Regresión lineal y variantes

Los modelos de **regresión lineal** buscan representar la dependencia entre un **atributo** y la **variable objetivo** a través de una **linea recta**, elegida con el fin de minimizar la distancia entre esta y todas las instancias del conjunto de datos - **reducir el error** ajustando los parámetros de la recta, buscando minimizar la **suma residual de errores cuadrados** en un proceso de optimización conocido como **mínimos cuadrados**.

Si bien el algoritmo inicial está propuesto para un único atributo, es posible **generalizarlo a múltiples atributos** - transformando la linea en un hiperplano, pero manteniendo el resto del proceso de optimización.

Estos modelos son **simples y muy eficientes** a la hora de ser entrenados y utilizados durante la inferencia. Ahora bien, también presentan el problema de que esta simpleza puede causar una **falta de capacidad computacional** a la hora de aprender relaciones más complejas entre datos, y **peor rendimiento cuando no existe independencia entre los atributos** - aunque existen diversas propuestas para solventar estos problemas.

#### - Regresión lineal

El modelo más simple de regresión lineal es el descrito previamente, implementado en `scikit-learn` en la clase `LinearRegression`. 

Debido a la simplicidad de este modelo, **no contiene ningún hiperparámetro a ajustar**.

In [18]:
# LINEAR REGRESSION
model_name = "linear_regression"

# No hyperparameters
models_dict[model_name] = {
    "hyperparameters": None,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        ("regression", LinearRegression())
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - Ridge (L2)

**Ridge**, también conocido como **regresión lineal L2**, es una variante del modelo original de regresión lineal que añade un **factor de penalización $\lambda$** al error a optimizar, con el fin de **reducir la complejidad final del modelo** - entendiendo la complejidad como **la media del valor cuadrado de los coeficientes** - penalizando los atributos con parámetros excesivamente altos.

De esta forma, se obtienen modelos con **coeficientes más pequeños** a mayores valores de $\alpha$ - llevando a un modelo **más robusto ante variables correlacionadas** y **más resistente al sobreajuste**. Ahora bien, el modelo en general **no reduce el número de atributos utilizados**.

El modelo (`Ridge` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.
    - Se probarán distintos **ordenes de magnitud** en el rango $[1*10^{-6}/1*10^6]$

In [19]:
# LINEAR REGRESSION - L2
model_name = "ridge_l2"
hyperparameter_grid = {
    "alpha": [1*10**num for num in range(-6, 7)]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        ("regression", Ridge())
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - Lasso (L1)

***Lasso (Least Absolute Shrunkage and Selection Operator)***, también conocido como **regresión lineal L1**, es una variante del modelo original de regresión lineal que añade un **factor de penalización $\lambda$** al error a optimizar, buscando reducir la **complejidad del modelo** - de forma muy similar a *Ridge*.

La diferencia principal entre **Ridge** y **Lasso** reside en como se calcula esta complejidad - pasando en este caso a medirla como la **media del valor absoluto de los coeficientes** frente al valor cuadrado. Esto se traduce en un modelo que **filtra los atributos menos relevantes** reduciendo sus coeficientes a 0 - pero más sensible a las **variables correlacionadas**.

El modelo (`Lasso` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.
    - Se probarán distintos **ordenes de magnitud** en el rango $[1*10^{-6}/1*10^6]$

In [20]:
# LINEAR REGRESSION - L1
model_name = "lasso_l1"
hyperparameter_grid = {
    "alpha": [1*10**num for num in range(-6, 7)]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        ("regression", Lasso())
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - *Elastic-Net* (L1 y L2)

**Elastic-Net** es una combinación de los dos modelos de regresión lineal descritos previamente - **Ridge** (L1) y **Lasso** (L2) - donde se aplican a la vez ambas penalizaciones a la complejidad del modelo de forma **proporcional a un ratio `r`** determinado como hiperparámetro.

De esta forma, el modelo es capaz de aprovechar el comportamiento de ambas aproximaciones - modelos **más resistentes al sobreajuste y a las variables correlacionadas** (Lasso) pero capaces de **filtrar variables irrelevantes** (Ridge).

El modelo (`ElasticNet` en `scikit-learn`) tiene los siguientes **hiperparámetros** a ajustar:
- **Alfa ($\alpha$)**: Constante que regula el **factor de penalización** a los parámetros. Valores más altos indican **penalizaciones más altas**.
    - Se probarán distintos **ordenes de magnitud** en el rango $[1*10^{-6}/1*10^6]$
- **Ratio (`l1_ratio`)**: Valor en el rango $[0, 1]$ que determina **el ratio en el que se aplica la penalización de Ridge**. Concretamente:
    - `0` significa un modelo **Lasso (L2)**.
    - `1` significa un modelo **Ridge (L1)**.
    - Cualquier otro número significa una combinación, donde **números mayores** indican mayor influencia de Ridge, y **números menores** indican mayor influencia de Lasso.

In [21]:
# LINEAR REGRESSION - ELASTIC-NET
model_name = "elastic_net"
hyperparameter_grid = {
    "alpha": [1*10**num for num in range(-6, 7)],
    "l1_ratio": [0.25, 0.5, 0.75]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        ("regression", ElasticNet())
    ])
    models_dict[model_name]["models"][key] = pipeline

---

### 4.1.2. Árboles de decisión

Los modelos de **árboles de decisiones** buscan representar el conocimiento aprendido por el modelo a través de una **estructura de reglas jerárquica en forma de arbol** dividida en **nodos**, **ramas** y **hojas**; donde:
- **Nodo**: Un nodo interno del arbol - no final - donde se **comprueba el valor de un atributo**. El nodo se **ramifica** en tantas ramas como posibles resultados tenga la comprobación.
- **Rama**: Tras la comprobación, cada **rama** representa un posible resultado de la comprobación, generalmente un **posible valor del atributo** - para atributos categóricos - o **si el atributo es mayor o menos a un umbral** - para atributos numéricos. Esta rama puede llevar a otro nodo, donde se repetiría el proceso, o a una **hoja**.
- **Hoja**: Un **valor final predicho** para la variable objetivo - el valor estimado para una combinación de **atributos y sus valores**, representados por los nodos y ramas tomados para llegar hasta la hoja.

Por tanto, el objetivo de un algoritmo de aprendizaje de árboles de decisión es aprender el **conjunto de reglas** - concretamente, los **nodos**, **ramificaciones** y **hojas** que contiene el arbol - para representar de la forma más precisa posible el conjunto de datos.

En general, los árboles de decisión son modelos muy utilizadospor su **simpleza a la hora de interpretarlos** y a su **facilidad** para ser aplicados a diversos problemas. Ahora bien, **pueden conducir al sobreajuste** si no se eligen con cuidado los hiperparámetros y son **inestables** ante variaciones pequeñas en el conjunto de datos. 

Por esto, generalmente suelen ser utilizados como modelos más sencillos dentro de **ensembles** - como se verá en el siguiente apartado.

---

La implementación de los árboles de decisión para regresión en `scikit-learn` se encuentra en la clase `DecisionTreeRegressor`. Esta clase toma los siguientes **hiperparámetros**:
- **Profundidad máxima (`max_depth`)**: Profundidad máxima del arbol. A mayor profundidad, más complejas pueden ser las divisiones aprendidas - pero más propenso es a sobreajustar el modelo.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.
- **Número mínimo de instancias por hoja (`min_samples_leaf`)**: Un nodo nunca se particiona si las hojas resultantes tienen menos instancias de las especificadas.
- **Criterio de particionamiento (`criterion`)**: Las particiones de los nodos se seleccionan buscando la **partición que minimiza el error resultante**. Las opciones a considerar son:
    - `squared_error`: Error cuadrático medio.
    - `friedman_mse`: Error absoluto medio utilizando una **corrección de Friedman** para considerar además las **probabilidades** de las particiones resultantes.
    - `absolute_error`: Error absoluto medio.

In [22]:
# DECISION TREE
model_name = "decision_tree"
hyperparameter_grid = {
    "max_depth": [2, 5, 8, 10, 20, None],
    "min_samples_split": [2, 5, 10, 20, 50],
    "min_samples_leaf": [1, 2, 5, 10, 20, 50],
    "criterion": ["squared_error", "friedman_mse", "absolute_error"]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", DecisionTreeRegressor(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

---

### 4.1.3. Máquinas de vectores de soporte

Los modelos de **máquina de vector de soporte**, de forma similar a los modelos de regresión lineal, buscan encontrar un **hiperplano** que sea capaz de dividir el conjunto de datos de forma óptima. Ahora bien, en este caso se busca que el hiperplano tenga un **márgen respecto a los vectores de soporte** - las instancias más cercanas al hiperplano - cuya funcionalidad depende del tipo de problema a resolver:
- **Clasificación**: El caso más típico, se busca un **márgen máximo** para distinguir las clases del conjunto de datos - se busca el hiperplano **más genérico** capaz de distinguir las instancias del conjunto de datos.
- **Regresión**: En este caso, el margen cumple la idea contraria - las instancias **dentro del margen** no son relevantes para el modelo, al considerarse que son suficientemente cercanas a la predicción del modelo. En este caso, se busca encontrar el hiperplano que **minimice la distancia de todas las instancias a su margen** - estableciendo este margen en base a un **error mínimo $\epsilon$**.

La otra gran particularidad de estos modelos son los **métodos kernel**. En ocasiones, no es posible encontrar un hiperplano que sea capaz de cumplir las restricciones impuestas - o dividir de forma limpia el conjunto de datos o minimizar la distancia a las instancias. En estos casos, las máquinas de vector soporte son capaces de **proyectar las instancias del conjunto de datos en dimensionalidades más altas** a traves de lo que se conocen como **funciones Kernel** - buscando, de esta forma, un hiperplano de mayor dimensionalidad capaz de cumplir las condiciones necesarias.

En general, las máquinas de vectores de soporte son modelos **efectivos en problemas de alta dimensionalidad** y **eficientes con los datos** - al centrarse principalmente en los vectores de soporte que definen los márgenes del hiperplano. Ahora bien, son modelos **lentos de entrenar debido a su complejidad** y **propensos al sobreajuste** si no se cuidan sus hiperparámetros.

---

Existen varias implementaciones de estos modelos en `scikit-learn`. En concreto, se van a considerar **dos**:

#### - `LinearSVR`

La clase `LinearSVR` implementa una máquina de vectores de soporte para regresión optimizada para utilizar una **función kernel lineal** - una implementación más rápida pero limitada únicamente a este tipo de Kernel.

Esta clase toma los siguientes hiperparámetros:
- **Epsilon (`epsilon`):** Margen de error del hiperplano. En general, las instancias que se encuentran a menos de $\epsilon$ del hiperplano **se ignoran a la hora de optimizar el error**.
- **Tolerancia (`tol`)**: Tolerancia durante el entrenamiento. Una vez el error sea menor a la tolerancia, el proceso de entrenamiento se detiene.
- **Parametro de regularización (`C`)**: La potencia de la regularización realizada por el modelo es **inversamente proporcional** al valor de `C`.

In [23]:
# SUPPORT VECTOR MACHINE - LINEAR
model_name = "linear_svr"
hyperparameter_grid = {
    "epsilon": [0, 1e-3, 1e-2, 1e-1, 1, 10],
    "tol": [1e-6, 1e-5, 1e-4, 1e-3],
    "C": [0.1, 1, 5, 10, 50, 100]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", LinearSVR(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - `SVR`

La clase `SVR` implementa una máquina de vectores de soporte para regresión general - capaz de utilizar **diversas funciones kernel no lineares** a costa de un coste computacional más elevado.

Esta clase toma los siguientes hiperparámetros:
- **Función kernel (`kernel`):** Función kernel a utilizar por el algoritmo. En concreto, se van a considerar las siguientes:
    - `poly`: Función **polinómica**. Contiene un hiperparámetro adicional - `degree`, el **grado del polinomio**.
    - `rbf`: Función **gaussiana**.
    - `sigmoid`: Función **sigmoide**.
- **Epsilon (`epsilon`):** Margen de error del hiperplano. En general, las instancias que se encuentran a menos de $\epsilon$ del hiperplano **se ignoran a la hora de optimizar el error**.
- **Tolerancia (`tol`)**: Tolerancia durante el entrenamiento. Una vez el error sea menor a la tolerancia, el proceso de entrenamiento se detiene.
- **Parametro de regularización (`C`)**: La potencia de la regularización realizada por el modelo es **inversamente proporcional** al valor de `C`.

Debido a los hiperparámetros distintos utilizados por cada función kernel, se proponen **tres modelos separados** - uno para cada función:

##### - *SVR polinómica*

In [24]:
# SUPPORT VECTOR MACHINE - POLYNOMIC
model_name = "poly_svr"
hyperparameter_grid = {
    "degree": [2, 3, 4, 5],
    "epsilon": [0, 1e-3, 1e-2, 1e-1, 1, 10],
    "tol": [1e-6, 1e-5, 1e-4, 1e-3],
    "C": [0.1, 1, 5, 10, 50, 100]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", SVR(kernel="poly"))
    ])
    models_dict[model_name]["models"][key] = pipeline

##### - *SVR gaussiana*

In [25]:
# SUPPORT VECTOR MACHINE - GAUSSIAN
model_name = "rbf_svr"
hyperparameter_grid = {
    "epsilon": [0, 1e-3, 1e-2, 1e-1, 1, 10],
    "tol": [1e-6, 1e-5, 1e-4, 1e-3],
    "C": [0.1, 1, 5, 10, 50, 100]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", SVR(kernel="rbf"))
    ])
    models_dict[model_name]["models"][key] = pipeline

##### - *SVR sigmoide*

In [26]:
# SUPPORT VECTOR MACHINE - SIGMOIDAL
model_name = "sigmoid_svr"
hyperparameter_grid = {
    "epsilon": [0, 1e-3, 1e-2, 1e-1, 1, 10],
    "tol": [1e-6, 1e-5, 1e-4, 1e-3],
    "C": [0.1, 1, 5, 10, 50, 100]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", SVR(kernel="sigmoid"))
    ])
    models_dict[model_name]["models"][key] = pipeline

---

<a id="section4-2"></a>

## 4.2. *Ensembles* - Estimadores basados en agrupaciones de estimadores simples

En esta sección se definen los *ensembles* a utilizar durante la experimentación. 

Un ***ensemble*** es un algoritmo de aprendizaje automático que utiliza una **agrupación de modelos sencillos** - generalmente **con menor complejidad** que si se utilizasen de forma independiente - para su funcionamiento. Estos modelos son entrenados sobre el conjunto de datos - frecuentemente sobre **subconjuntos aleatorios de instancias y atributos** -, y el resultado del *ensemble* se obtiene como una **agrupación** de los resultados de todos los modelos - generalmente mediante algún tipo de **ponderación**.

Dependiendo de cómo se utilizan los modelos simples, se pueden identificar dos grandes familias de *ensembles*:
- **Bagging (Boostrap Aggregating)**: Cada modelo se entrena de forma **independiente** al resto.
- **Boosting**: Los modelos se entrenan de forma **secuencial** - donde el error de un modelo afecta a cómo se entrena el siguiente modelo.

Los modelos de *ensemble*, junto a modelos de **aprendizaje profundo** - como las **redes neuronales** - son considerados el **estado del arte** del aprendizaje supervisado actualmente. Es de esperar que sus resultados superen al resultado de los modelos de *baseline*.

---

### 4.2.1. *Bagging (Boostrap Aggregating)*

Los modelos de **bagging (bootstrap aggregating)** son agrupaciones de modelos de aprendizaje automático simples, donde cada modelo se entrena sobre **un subconjunto aleatorio de instancias del conjunto de datos** mediante un proceso conocido como **bootstrapping** (un muestreo aleatorio uniforme con reemplazo). El resultado final del *ensemble* consiste en la **unión** de los resultados de cada uno de los modelos subyacentes - ya sea por **votación**, eligiendo el resultado mayoritario, o por **promedio** de los resultados.

La principal meta de esta técnica es **reducir la varianza y el sobreajuste** de los modelos simples subyacentes - al estar cada modelo expuesto a un subconjunto aleatorio de datos con posibles duplicados, el conjunto de los modelos entrenados va a tener una mayor **diversidad**, permitiendo al *ensemble* ser capaz de **generalizar** de forma más robusta.

En general, las técnicas de *bagging* obtienen modelos más **robustos** debido a su resistencia al sobreajuste. Además, son facilmente **paralelizables** - al entrenarse de forma independiente cada modelo - y **compatibles con cualquier modelo subyacente**, aunque funcionan mejor al utilizarse con modelos propensos a sobreajustar como los **árboles de decisión**.

Se van a probar concretamente **dos** modelos de *bagging* usados con frecuencia en la actualidad - si bien sería posible crear un *ensemble* con cualquier modelo.

#### - *Random Forests*

***Random Forest*** es un modelo de aprendizaje automático basado en *ensembles* de **bagging**, consistente en un conjunto de **arboles de decisión profundos** donde cada arbol se entrena a la vez sobre un **subconjunto aleatorio de datos** y un **subconjunto aleatorio de los atributos** - siendo los atributos aleatorios la principal diferencia con el resto de modelos de *bagging*.

Al introducir estas dos fuentes de aleatoriedad, cada arbol **sobreajusta** a una parte distinta del conjunto de datos - teniendo, en conjunto, un grupo de **arboles de decisión profundos especializados** capaces de obtener un resultado **más generalizado** a través de un promedio de todas las salidas.

---

El modelo está implementado en `scikit-learn` a través de la clase `RandomForestRegressor`. Si bien se pueden ajustar los hiperparámetros de igual manera que con los arboles de decisiones individualmente, se van a considerar los **siguientes hiperparámetros**:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Número de atributos considerados por arbol (`max_features`):** Dependiendo del valor, se considera el siguiente número de atributos por arbol:
    - `None`: Cada arbol tiene acceso a **todos los atributos**.
    - `sqrt`: Cada arbol tiene acceso a un subconjunto de tamaño equivalente a **la raiz cuadrada del número de atributos**.
    - **Valor numérico**: Cada arbol tiene acceso al **porcentaje indicado del número de atributos**. En general, **0.3** es un valor utilizado con frecuencia en la literatura.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de cada arbol entrenado.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.

In [29]:
# RANDOM FOREST
model_name = "random_forest"
hyperparameter_grid = {
    "n_estimators": [50, 100, 150],
    "max_features": [None, "sqrt", 0.3],
    "max_depth": [None, 5, 10, 25],
    "min_samples_split": [2, 10, 25, 50]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", RandomForestRegressor(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - *Extremely Randomized Trees (Extra Trees)*

***Extremely Randomized Trees*** (también conocido como ***Extra Trees***) es una variación del modelo de *Random Forest* que añade un tercer factor de aleatoriedad a la forma en la que se construyen los árboles.

Durante el proceso de construcción del arbol, en cada nodo se elige el atributo que **más reduce el error a la hora de particionar el conjunto de datos** - comprobando, para cada posible atributo, el **umbral de partición** que más reduce el error. En el caso de ***Extra Trees***, en vez de elegir el mejor umbral se elige **un umbral aleatorio para cada atributo** - eligiendo, después, el **atributo con umbral aleatorio** que más reduce el error.

De esta manera se **reduce más la varianza del modelo final** al tener árboles más aleatorios - a riesgo de **aumentar el sesgo final**.

---

El modelo está implementado en `scikit-learn` a través de la clase `ExtraTreesRegressor`. Los hiperparámetros a ajustar son, esencialmente, los mismos que se han ajustado en *Random Forest*:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Número de atributos considerados por arbol (`max_features`):** Dependiendo del valor, se considera el siguiente número de atributos por arbol:
    - `None`: Cada arbol tiene acceso a **todos los atributos**.
    - `sqrt`: Cada arbol tiene acceso a un subconjunto de tamaño equivalente a **la raiz cuadrada del número de atributos**.
    - **Valor numérico**: Cada arbol tiene acceso al **porcentaje indicado del número de atributos**. En general, **0.3** es un valor utilizado con frecuencia en la literatura.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de cada arbol entrenado.
- **Número mínimo de instancias para particionar un nodo (`min_samples_split`)**: Para poder particionar un nodo - y que no sea una hoja -, este nodo debe tener al menos el número de instancias indicado.

In [35]:
# EXTREMELY RANDOM TREES (EXTRA TREES)
model_name = "extra_trees"
hyperparameter_grid = {
    "n_estimators": [50, 100, 150],
    "max_features": [None, "sqrt", 0.3],
    "max_depth": [None, 5, 10, 25],
    "min_samples_split": [2, 10, 25, 50]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", ExtraTreesRegressor(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

---

### 4.2.2. *Boosting*

Los modelos de ***Boosting*** son agrupaciones de modelos de aprendizaje automático **debiles** (entendiendose como *debil* a un modelo con poca capacidad computacional), donde los modelos se entrenan de forma **secuencial** - de forma que los errores de un modelo son considerados en el siguiente modelo mediante **importancias asignadas a cada instancia del conjunto de datos**, teniendo las instancias con mayor error un mayor peso a la hora de entrenar los modelos. El resultado final del *ensemble* consiste en una **ponderación** de los resultados de todos los modelos - donde los modelos con menor error tienen más peso en la votación final.

A diferencia de **bagging** - donde cada modelo se entrena en paralelo sobre un subconjunto aleatorio de atributos -, **boosting** entrena los modelos sobre el conjunto de datos completo ponderado.

La principal meta de esta técnica es **crear un conjunto de datos capaz de identificar relaciones complejas en el conjunto de datos** - al entrenar cada modelo teniendo en cuenta los errores producidos por el modelo anterior, se acaban produciendo modelos **muy especializados** para errores específicos. Además, la **agrupación ponderada** de los resultados permite al *ensemble* obtener resultados **sustancialmente mejores** a los de los modelos individuales.

Las técnicas de **boosting** tienden a obtener **mejores resultados** tanto en varianza como en sesgo - aunque suelen ser modelos **más caros y lentos de entrenar** (al entrenarse de forma secuencial) y **más susceptibles a ruido y valores extremos** en el conjunto de datos.


---

Se considera **un modelo clásico de boosting** - actualmente en desuso a favor de otras técnicas que se verán posteriormente:

#### - *Adaptive Boosting (AdaBoost)*

***Adaptive Boosting*** (también conocido como ***AdaBoost***) es el modelo que introdujo el concepto de *boosting* y ha sido, durante muchos años, uno de los principales *ensembles* utilizados en el proceso de ciencia de datos.

*AdaBoost* entrena de forma secuencial **modelos débiles** - concretamente **tocones**, arboles de decisión de baja profundidad - sobre el **conjunto de datos completo**. Ahora bien, cada instancia del conjunto de datos tiene una **importancia** ponderada por su peso, de forma que:
- Tras cada iteración, el peso de las instancias se ajusta -  las **instancias predecidas con mayor error aumentan su importancia**, por lo que **las instancias más dificiles de predecir** tienen los pesos más elevados.
- Cuando se entrena un modelo, la importancia de cada instancia se pondera por su peso - por lo que **los modelos se sesgan hacia las instancias más dificiles de clasificar**.

Además, **cada modelo tiene un peso asociado** en base a su error total. El resultado final del modelo es una **agregación ponderada** de todos los resultados - donde los modelos con menor error tienen una mayor importancia en la ponderación final.

---

El modelo está implementado en `scikit-learn` en la clase `AdaBoostRegressor`. Si bien nos deja elegir cualquier modelo como estimador base, se utilizará el modelo por defecto (un **arbol de decisión de profundidad 3**). Se consideran los siguientes hiperparámetros:
- **Número de estimadores (`n_estimators`):** Número de árboles a considerar en el modelo.
- **Ratio de aprendizaje (`learning_rate`):** Ponderación aplicada a cada modelo nuevo - en general, representa la **velocidad** a la que el modelo puede aprender, donde **valores más altos** se traducen en cambios más rápidos y **valores más bajos** se traducen en cambios más lentos.

In [32]:
# ADAPTIVE BOOSTING (ADABOOST)
model_name = "adaboost"
hyperparameter_grid = {
    "n_estimators": [50, 100, 150, 200, 250],
    "learning_rate": [1e-4, 1e-3, 1e-2, 1e-1, 1]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", AdaBoostRegressor(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

---

### 4.2.3. *Gradient Boosting*

Los modelos de ***Gradient Boosting*** son variaciones de los modelos de **boosting** en los que:
- Se utiliza **gradiente descendiente** a la hora de calcular la importancia de las instancias del conjunto de datos - en vez de actualizar directamente los pesos en base a las **residuales**, se calcula el **gradiente** que maximiza la reducción de una **función de error genérica** para cada instancia - ponderando el entrenamiento del modelo nuevo utilizando este gradiente.
- Si bien se siguen usando modelos débiles, **se tiende a usar modelos más complejos que en las técnicas tradicionales**.

Realmente, los modelos de *Gradient Boosting* son **generalizaciones** de los modelos originales de *boosting* (principalmente ***AdaBoost***) para poder utilizar **cualquier función de error genérica** - con la única restricción de necesitar una función **derivable** para poder usarse durante gradiente descendiente.

Por lo general, los modelos de *gradient boosting* ofrecen mejores resultados que las técnicas tradicionales de *boosting* - a costa de un mayor coste computacional para su entrenamiento.

---

Se van a considerar **cuatro** modelos de *Gradient Boosting* - actualmente **estado del arte** para los problemas de aprendizaje automático sobre datos tabulares estructurados. En general, las diferencias entre estos modelos están más centradas en la **implementación técnica de los modelos** - aunque también existen algunas diferencias a la hora de definir los algoritmos:

#### - *Extreme Gradient Boosting (XGBoost)*

***eXtreme Gradient Boosting*** (también conocido como ***XGBoost***) es un modelo y una librería de código abierto creada con el objetivo de crear implementaciones **escalables, portables y capaces de ser distribuidas entre distintas máquinas**.

Concretamente, ***XGBoost*** es un algoritmo de *Gradient Boosting* que utiliza **árboles de decisión** como modelos base, con las siguientes características:
- En vez de utilizar **gradiente descendiente**, se utiliza el ***método de Newton-Raphson** para optimizar el error - utilizando **segundas derivadas** para calcular el gradiente que minimiza el error de los modelos.
- Los árboles se entrenan de forma **paralela** - pudiendo distribuir el entrenamiento entre varios computadores a la vez.

---

La implementación de ***XGBoost*** está disponible en la librería `xgboost` en la clase `XGBRegressor`, aunque es totalmente compatible con `scikit-learn` para preprocesamiento y experimentación. Se consideran los siguientes parámetros:
- **Ratio de aprendizaje (`learning_rate`):** Factor por el que se **multiplican las actualizaciones de pesos** para ralentizar el aprendizaje.
- **Profundidad máxima (`max_depth`):** Profundidad máxima de los árboles entrenados.
- **Umbral de mejora para partición (`gamma`)**: Es necesario que se reduzca el error al menos `gamma` para que se considere una partición.
- **Ratio de muestreo del conjunto de datos (`subsample`):** Porcentaje del conjunto de entrenamiento que se muestrea para entrenar cada arbol.
- **Ratio de muestreo de los atributos (`colsample_bytree`):** Porcentaje del conjunto de atributos que se muestrea para entrenar cada arbol.

Existe una gran cantidad de posibles hiperparámetros a ajustar, por lo que se ha optado por utilizar un **subconjunto reducido** de estos.

In [ ]:
# EXTREME GRADIENT BOOST (XGBoost)
model_name = "xgboost"
hyperparameter_grid = {
    "learning_rate": Real(0, 1),
    "max_depth": Integer(4, 10),
    "gamma": Real(0, 1e4),
    "subsample": Real(0, 1),
    "colsample_bytree": Real(0, 1)
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", XGBRegressor(random_state=RANDOM_SEED, device="gpu"))
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - *Categorical Boosting (CatBoost)*

#### - *Light Gradient Boosting Model (LGBM)*

***Light Gradient Boosting Model*** (o ***LGBM***)

In [ ]:
# ADAPTIVE BOOSTING (ADABOOST)
model_name = "adaboost"
hyperparameter_grid = {
    "n_estimators": [50, 100, 150, 200, 250],
    "learning_rate": [1e-4, 1e-3, 1e-2, 1e-1, 1]
}

models_dict[model_name] = {
    "hyperparameters": hyperparameter_grid,
    "models": {}
}

# Create the pipelines and store them within the dictionary
for key in preprocessing_dict:

    pipeline = Pipeline([
        ("preprocessing", preprocessing_dict[key]),
        # Ensure that random states are set for models with random elements
        ("regression", AdaBoostRegressor(random_state=RANDOM_SEED))
    ])
    models_dict[model_name]["models"][key] = pipeline

#### - *Gradient Boosting* basado en histogramas (*HistGradientBoost*)

---

<a id="section5"></a>

# 5. Experimentación

- HAZ QUE LAS MÁQUINAS DE VECTOR SOPORTE UTILICEN RANDOMIZEDSEARCHCV EN VEZ DE GRIDSEARCHCV PARA EVITAR EXCESO DE TIEMPO
- LA MAYORIA DE MODELOS INCORPORAN ONE HOT AUTOMATICAMENTE:
    - XGBOOST
    - CATBOOST
- CATBOOST NO TIENE QUE TENER ONE HOT ENCODING
- XGBOOST PUEDE TENER EARLY STOPPING CON VALIDACION - HAY QUE INDICAR GPU Y TENER CUIDADO CON EL THREADING
- USAR BAYESIAN SEARCH?

<a id="section6"></a>

# 6. Análisis de resultados